In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
from tensorflow import keras
import matplotlib.image as mpimg
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Model
from keras.models import clone_model
from keras.layers import Dense,Conv2D,Flatten,MaxPool2D,Dropout,BatchNormalization, Input
from keras.optimizers import Adam
from keras.callbacks import ReduceLROnPlateau
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
import PIL.Image as Image, PIL.ImageDraw as ImageDraw, PIL.ImageFont as ImageFont
from matplotlib import pyplot as plt
import seaborn as sns
import cv2
import gc

In [ ]:
sample_sub = pd.read_csv('/kaggle/input/bengaliai-cv19/sample_submission.csv')

In [ ]:
sample_sub.head()

In [ ]:
class_map=pd.read_csv('/kaggle/input/bengaliai-cv19/class_map_corrected.csv')
class_map.head()

In [ ]:
img_size=64
channels=1

In [ ]:
from keras.models import load_model

In [ ]:
model=load_model('/kaggle/input/my-model/model_final.h5')

In [ ]:
submission=sample_sub

In [ ]:
count=0

In [ ]:
submission.head()

In [ ]:
for j in range(4):
    test_df = pd.read_parquet(f'/kaggle/input/bengaliai-cv19/test_image_data_{j}.parquet',engine='pyarrow')
    X_test = test_df.drop(['image_id'], axis=1)
    X_test=np.array(X_test).reshape(-1,137,236)
    x=X_test.shape[0]
    test_images=[]
    for i in range(x):
        image=X_test[i]
        image=cv2.resize(image,(img_size,img_size),interpolation=cv2.INTER_AREA)
        test_images.append(image)
    del X_test
    gc.collect()
    test_images=np.array(test_images).reshape(-1,64,64,1)

    y=model.predict(test_images)
    graphemes=np.argmax(y[0],axis=1)
    vowel=np.argmax(y[1],axis=1)
    consonants=np.argmax(y[2],axis=1)
    for i in range(len(test_images)):
        submission.loc[count,'target']=consonants[i]
        count+=1
        submission.loc[count,'target']=graphemes[i]
        count+=1
        submission.loc[count,'target']=vowel[i]
        count+=1
    del graphemes
    del vowel
    del consonants
    del test_images

In [ ]:
submission.shape

In [ ]:
submission.to_csv('/kaggle/working/submission.csv',index=False)